In [23]:
%run Requirements.ipynb

In [24]:
%run secrets.ipynb

In [25]:
def get_playlist_tracks(access_token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    return response.json().get("items", [])

def extract_detailed_info(tracks):
    data = []
    seen = set()
    for item in tracks:
        track = item['track']
        track_id = track['id']
        track_name = track['name']
        album = track['album']
        album_id = album['id']
        album_name = album['name']
        artists = track['artists']
        for artist in artists:
            artist_id = artist['id']
            artist_name = artist['name']
            unique_key = (artist_id, album_id, track_id)
            if unique_key not in seen:
                seen.add(unique_key)
                data.append({
                    'ArtistName': artist_name,
                    'artist_id': artist_id,
                    'album': album_name,
                    'album_id': album_id,
                    'track': track_name,
                    'track_id': track_id
                })
    return data


access_token = AT
tracks = get_playlist_tracks(access_token, Secrets.get('playlist_id'))
detailed_info = extract_detailed_info(tracks)

df = spark.createDataFrame(detailed_info)
df.write.mode('overwrite').format('csv').options(header=True).save('Staging/TopGlobalArtists50')
df = pd.DataFrame(detailed_info)
df.to_csv("TopGlobalArtists50.csv", header=True)